In [1]:
import numpy as np

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers, models, optimizers, callbacks, losses
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [ ]:
# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
# Define the labels
labels = np.arange(10)

# Plot a sample image from each label
fig, axs = plt.subplots(1, 10, figsize=(15, 1))
for i, label in enumerate(labels):
    ax = axs[i]
    img = x_train[y_train[:, 0] == label][0]
    ax.imshow(img)
    ax.axis("off")
    ax.set_title(str(label))
plt.show()

In [ ]:
# Normalize pixel values to be between 0 and 1
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Convert labels to categorical one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

In [ ]:
# Define the data augmentation pipeline
datagen = image.ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest'
)

In [ ]:
# Define the CNN model architecture
model = models.Sequential([
    layers.Input(shape=(32, 32, 3)),

    layers.Conv2D(filters=96, kernel_size=(3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(filters=96, kernel_size=(3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(filters=96, kernel_size=(3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Conv2D(filters=192, kernel_size=(3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(filters=192, kernel_size=(3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(filters=192, kernel_size=(3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Conv2D(filters=192, kernel_size=(3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(filters=192, kernel_size=(1, 1), padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(filters=10, kernel_size=(1, 1), padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),
    
    layers.GlobalAvgPool2D(),
])

model.summary()

In [ ]:
model.load_weights("cifar10_cnn_model.h5")

In [ ]:
optimizer = optimizers.Adam()

In [ ]:
# Compile the model
model.compile(loss=losses.CategoricalCrossentropy(from_logits=True), optimizer=optimizer, metrics=["accuracy"])

In [ ]:
model.load_weights("cifar10_cnn_model.h5")

In [ ]:
# Define early stopping and checkpoint callbacks
early_stopping_cb = callbacks.EarlyStopping(patience=10, restore_best_weights=True, verbose=True)
checkpoint_cb = callbacks.ModelCheckpoint("cifar10_cnn_model.h5", save_best_only=True, verbose=True)

In [ ]:
# Train the model with data augmentation
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=256),
    epochs=500,
    validation_data=(x_val, y_val),
    callbacks=[
        early_stopping_cb,
        checkpoint_cb
    ]
)

In [ ]:
# Plot the training and validation loss
plt.figure(figsize=(8, 6))
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

# Plot the training and validation accuracy
plt.figure(figsize=(8, 6))
plt.plot(history.history["accuracy"], label="Training Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [18]:
# Evaluate the model on the test set
score = model.evaluate(x_test, y_test)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

313/313 [==============================] - 7s 12ms/step - loss: 0.4138 - accuracy: 0.9119
Test loss: 0.4137522876262665
Test accuracy: 0.911899983882904


In [19]:
# Evaluate the model on the test set
score = model.evaluate(x_train, y_train)
print("Train loss:", score[0])
print("Train accuracy:", score[1])

1407/1407 [==============================] - 16s 11ms/step - loss: 0.0424 - accuracy: 0.9907
Train loss: 0.042400091886520386
Train accuracy: 0.9906666874885559


In [20]:
model.save("cifar10_cnn_model.h5")

In [2]:
model = models.load_model("cifar10_cnn_model.h5")

In [ ]:
# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values to be between 0 and 1
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Convert labels to categorical one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [ ]:
# Evaluate the model on the test set
score = model.evaluate(x_test, y_test)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [ ]:
# Evaluate the model on the test set
score = model.evaluate(x_train, y_train)
print("Train loss:", score[0])
print("Train accuracy:", score[1])

In [3]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_19 (Conv2D)          (None, 32, 32, 96)        2688      
                                                                 
 batch_normalization_19 (Bat  (None, 32, 32, 96)       384       
 chNormalization)                                                
                                                                 
 re_lu (ReLU)                (None, 32, 32, 96)        0         
                                                                 
 conv2d_20 (Conv2D)          (None, 32, 32, 96)        83040     
                                                                 
 batch_normalization_20 (Bat  (None, 32, 32, 96)       384       
 chNormalization)                                                
                                                                 
 re_lu_1 (ReLU)              (None, 32, 32, 96)       